In [1]:
! pip install river


[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: C:\Users\elson\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from PredictScore import PredictorScore as predict
import os
import pickle
import pandas as pd
from ClassModelWeather import UserOutfitModel as weather_model
from ClassModelStyle import UserOutfitModelS as style_model
from ClassModelCombination import PredictCombinationScore as combi_model
import numpy as np

jika ingin membuat file baru dengan fresh, buatlah unique user agar file historikal user masih baru, jika user sudah terdaftar, dia akan memakai 

In [3]:
#change this
user=101

model making to predict

In [4]:
model_C=combi_model(user)
predictscore=predict(user)
model_W=weather_model(user)
model_S=style_model(user)


weather data making

In [5]:
#change this
weather_data={
    "top":'shirt',
    "bottom":'jeans',
    "outer":'none',
    "weather":'clouds',
}

printing the result of model weather

In [6]:
W_score=model_W.predict_outfit_rating(weather_data)
print('W score is : ',W_score)

W score is :  5.0


data model for style

In [7]:
#change this
style_data={
    "top":'shirt',
    "bottom":'jeans',
    "outer":'none',
    "styleTop": 'streetwear',
    "styleBottom":'casual',
    "styleOuter":'none',
    "styleUser" : 'formal',
}

In [8]:
S_score=model_S.predict_outfit_rating(style_data)
print('S score is : ',S_score)

S score is :  3.833694742099122


data for combination table

In [9]:
outer="none"
top="shirt"
bottom="jeans"

In [10]:
C_score=model_C.predict_score_C(outer,top,bottom)
print('C score is : ',C_score)

C score is :  3


combine the scores to create final predict score with predict algorithm

In [11]:
PredictScore=predictscore.predict_score(C_score,S_score,W_score)
print('Predict Score is :',PredictScore)

Predict Score is : 4.050108422629736


so predicted score is 4

let's add user rating

In [12]:
userRate=1 #change this

let's update the score and weights and look how much different it could get

In [13]:
# this will give new Score to update the models
new_C_score,new_S_score,new_W_score=predictscore.update_weights_and_scores(C_score,S_score,W_score,userRate)

In [14]:
print('New C score to change the model is :',new_C_score)
print('New S score to change the model is :',new_S_score)
print('New W score to change the model is :',new_W_score)

New C score to change the model is : 1.1957450393983189
New S score to change the model is : 2.552032130171206
New W score to change the model is : 3.578451000264754


let's make the model to learn base on new score to change

make the data to suits the learning

In [15]:
# print(weather_data)
weather_data_update=weather_data.copy()
weather_data_update['rating']=new_W_score
style_data_update=style_data.copy()
style_data_update['adjusted_rate']=new_S_score


In [16]:
print(weather_data_update)
print(style_data_update)

{'top': 'shirt', 'bottom': 'jeans', 'outer': 'none', 'weather': 'clouds', 'rating': 3.578451000264754}
{'top': 'shirt', 'bottom': 'jeans', 'outer': 'none', 'styleTop': 'streetwear', 'styleBottom': 'casual', 'styleOuter': 'none', 'styleUser': 'formal', 'adjusted_rate': 2.552032130171206}


In [17]:
model_C.update_score_C(outer,top,bottom,new_C_score)
model_S.update_online_model(style_data_update)
model_W.update_online_model(weather_data_update)

In [18]:

W_score=model_W.predict_outfit_rating(weather_data)
S_score=model_S.predict_outfit_rating(style_data)
C_score=model_C.predict_score_C(outer,top,bottom)
PredictScore=predictscore.predict_score(C_score,S_score,W_score)
print('Iteration 1 = Predict Score is :',PredictScore)

Iteration 1 = Predict Score is : 2.7308575086619182


In [19]:
for x in range(5):
    new_C_score,new_S_score,new_W_score=predictscore.update_weights_and_scores(C_score,S_score,W_score,userRate)
    weather_data_update=weather_data.copy()
    weather_data_update['rating']=new_W_score
    style_data_update=style_data.copy()
    style_data_update['adjusted_rate']=new_S_score
    W_score=model_W.predict_outfit_rating(weather_data)
    S_score=model_S.predict_outfit_rating(style_data)
    C_score=model_C.predict_score_C(outer,top,bottom)
    PredictScore=predictscore.predict_score(C_score,S_score,W_score)
    print('Iteration',x+2,':','Predict Score is :',PredictScore)

Iteration 2 : Predict Score is : 2.2043103507196293
Iteration 3 : Predict Score is : 2.026946704417303
Iteration 4 : Predict Score is : 1.9729346340364946
Iteration 5 : Predict Score is : 1.9570993715509528
Iteration 6 : Predict Score is : 1.9525131185400064


bisa dilihat dari 5-6 iterasi,prediksi merunjuk ke satu, dan pelonjakan perubahan dari 4 ke 2 memperlihatkan semakin besar Error ( P - R ), semakin besar pula perubahan nya dan juga perubahan terbesar terjadi pada skor yang paling jauh terhadap skor Real